In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random

# Environment and Agent Setup
class AgentEnvironment:
    def __init__(self, num_targets=5, num_obstacles=3):
        self.agent_pos = np.array([0.0, 0.0])
        self.targets = [{"pos": np.random.rand(2) * 10, "weight": random.randint(1, 10)} for _ in range(num_targets)]
        self.obstacles = [np.random.rand(2) * 10 for _ in range(num_obstacles)]
        self.learning_rate = 0.05
        self.chaos = 0.2
        self.chaos_decay = 0.55

    def step(self):
        # Calculate target priorities
        target_scores = [
            t["weight"] / (np.linalg.norm(t["pos"] - self.agent_pos) + 1e-3)
            for t in self.targets
        ]
        # Select highest priority target
        best_target_idx = np.argmax(target_scores)
        target_pos = self.targets[best_target_idx]["pos"]

        # Move agent toward target
        direction = target_pos - self.agent_pos
        step = self.learning_rate * direction + self.chaos * np.random.randn(2)
        self.agent_pos += step

        # Check for target reach
        if np.linalg.norm(self.agent_pos - target_pos) < 0.5:
            self.targets.pop(best_target_idx)  # Remove target once reached

        # Reduce chaos over time
        self.chaos *= self.chaos_decay

    def run(self, steps=100):
        trajectory = [self.agent_pos.copy()]
        for _ in range(steps):
            if not self.targets:
                break
            self.step()
            trajectory.append(self.agent_pos.copy())
        return np.array(trajectory)

# Visualization
def plot_environment(env, trajectory):
    plt.figure(figsize=(8, 8))
    plt.plot(trajectory[:, 0], trajectory[:, 1], label="Trajectory")
    for target in env.targets:
        plt.scatter(*target["pos"], c="green", s=100 * target["weight"], label="Target")
    for obstacle in env.obstacles:
        plt.scatter(*obstacle, c="red", s=100, label="Obstacle")
    plt.scatter(*trajectory[0], c="blue", label="Start")
    plt.legend()
    plt.show()

# Initialize Environment
env = AgentEnvironment(num_targets=10, num_obstacles=5)
trajectory = env.run(steps=200)
plot_environment(env, trajectory)
